In [ ]:
from grid.refine import *

In [ ]:
keyed = keyed_arena()[['black_key', 'white_key', 'black_wins', 'white_wins']]
for b in [3, 5, 7, 9]:
    aux = database.pandas(f'mohex-{b}').reset_index()
    aux['black_key'] = f'{b}-' + aux.black_name
    aux['white_key'] = f'{b}-' + aux.white_name
    keyed = pd.concat([keyed, aux.reindex(columns=keyed.columns)])

black_wins = pd.pivot_table(keyed, 'black_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)
white_wins = pd.pivot_table(keyed, 'white_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)

wins = black_wins + white_wins.T
games = black_wins + white_wins + black_wins.T + white_wins.T

targets = set(keyed.black_key.values) | set(keyed.white_key.values)
targets = [t for t in targets if 'mohex' not in t]

with aljpy.parallel(solve) as p:
    solns = p.wait({t: p(t, games, wins) for t in targets})

solns = pd.DataFrame.from_dict(solns, orient='index')

aux = keyed_samples()
joint = pd.concat([aux, solns], 1)

In [ ]:
from grid.plots import *
joint['g'] = joint.index.str[:-2]
joint['flops'] = flops(joint)
joint['params'] = params(joint)

(ggplot(data=joint[joint.index.str[-2] != '0'])
    + geom_point(aes(x='flops', y='μ', group='g', color='boardsize'))
    + scale_x_continuous(trans='log10')
    + scale_color_continuous(trans='log10')
    + labs(title='slope is about the same in every board size')
    + mpl_theme(12, 8)
    + poster_sizes())

In [ ]:
joint

In [ ]:
joint = pd.concat([aux, solns], 1)